# Phase 3: The Two-Step Bayesian Modeling Framework

## Step 1: Short-Term Effects Model (UCM-MMM)

This model isolates the immediate, transitory impact of marketing activities on sales.

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scripts.mmm_optimized import UCM_MMM_Optimized

# Load the prepared data from previous notebook
df = pd.read_csv('../data/prepared_data.csv', parse_dates=['Date'])

print(f"Loaded {len(df)} weeks of data")
print(f"Date range: {df['Date'].min()} to {df['Date'].max()}")

# Extract data components
sales = df['revenue'].values
marketing_channels = ['Content Marketing', 'Events', 'Google Ads', 'LinkedIn']
marketing_data = df[marketing_channels].values

# Control variables (optional - improves model accuracy)
control_cols = ['Competitor_A_Spend', 'Competitor_B_Spend', 
                'GDP_Growth', 'Unemployment_Rate', 'Consumer_Confidence']
control_data = df[control_cols].values

print(f"\nMarketing channels: {marketing_channels}")
print(f"Control variables: {control_cols}")
print(f"\nSales summary:")
print(f"  Mean: ${sales.mean():,.0f}")
print(f"  Std: ${sales.std():,.0f}")

In [ ]:
# Fit the Model with MCMC

print("Fitting UCM-MMM with MCMC...")
print("Configuration:")
print("  • 500 tuning + 500 draws × 4 chains")
print("  • Target accept: 0.95 (very robust)")
print("  • This will take 15-25 minutes...")
print()

mmm.fit(
    draws=500,
    tune=500,
    chains=4,
    target_accept=0.95
)

print("\n✓ Model fitted successfully!")

# Check convergence
summary = mmm.summary()
rhat_max = summary['r_hat'].max()
ess_min = summary['ess_bulk'].min()

print(f"\nConvergence Diagnostics:")
print(f"  Max R-hat: {rhat_max:.4f} (should be < 1.01)")
print(f"  Min ESS:   {ess_min:.0f} (should be > 1000)")

if rhat_max < 1.01:
    print("  ✓ EXCELLENT convergence!")
elif rhat_max < 1.05:
    print("  ✓ Good convergence")
else:
    print("  ⚠ Consider increasing draws")

In [ ]:
# Visualize Adstock Parameters

import arviz as az

# Plot adstock parameters (alpha)
alpha_samples = mmm.trace.posterior['alpha'].values.reshape(-1, len(marketing_channels))

fig, ax = plt.subplots(figsize=(10, 6))
ax.violinplot(alpha_samples, positions=range(len(marketing_channels)), 
              showmeans=True, showmedians=True)
ax.set_xticks(range(len(marketing_channels)))
ax.set_xticklabels(marketing_channels, rotation=45, ha='right')
ax.set_ylabel('Adstock Parameter (α)', fontsize=12, fontweight='bold')
ax.set_title('Adstock Carryover Rates by Channel', fontsize=14, fontweight='bold', pad=20)
ax.set_ylim(0, 1)
ax.axhline(0.5, color='red', linestyle='--', alpha=0.3, label='α=0.5 (50% carryover)')
ax.legend()
ax.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

print("\nAdstock Interpretation:")
print("  α = 0.3 → Effects decay quickly (30% remains next week)")
print("  α = 0.5 → Moderate persistence (50% remains)")
print("  α = 0.7 → Strong carryover (70% remains)")

In [ ]:
# Plot adstock and saturation curves
mmm.plot_adstock_decay_rates()
mmm.plot_saturation_curves()

In [ ]:
# Calculate Short-Term ROI

short_term_roi = mmm.calculate_short_term_roi()

print("SHORT-TERM ROI (Immediate Activation)")
print("="*50)
for channel, roi in short_term_roi.items():
    print(f"{channel:20s}: ${roi:.2f} per $1 spent")

# Visualize
fig, ax = plt.subplots(figsize=(10, 6))
channels = list(short_term_roi.keys())
rois = list(short_term_roi.values())
colors = ['#2E86AB' if r > 0 else '#E63946' for r in rois]

ax.barh(channels, rois, color=colors, alpha=0.7)
ax.axvline(0, color='black', linewidth=1)
ax.set_xlabel('ROI per $1 Spent', fontsize=12, fontweight='bold')
ax.set_title('Short-Term Marketing ROI (Immediate Effects)', 
             fontsize=14, fontweight='bold', pad=20)
ax.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

print("\nNote: Short-term ROI captures only immediate sales impact.")
print("Long-term brand-building effects are measured in the next notebook.")

In [ ]:
# Calculate and plot channel ROI
mmm.plot_channel_roas()

In [ ]:
# Extract Base Sales for Long-Term Model

# Base sales = sales minus marketing effects
base_sales = mmm.get_base_sales()

# Save for next notebook
df_base = pd.DataFrame({
    'Date': df['Date'],
    'Base_Sales': base_sales
})
df_base.to_csv('../data/base_sales.csv', index=False)
print("✓ Base sales saved to data/base_sales.csv")

# Visualize decomposition
fig, ax = plt.subplots(figsize=(14, 6))

ax.plot(df['Date'], sales, 'o-', color='black', linewidth=2, 
        markersize=3, label='Actual Sales', alpha=0.7)
ax.plot(df['Date'], base_sales, '-', color='#2E86AB', linewidth=2, 
        label='Base Sales (without marketing)', alpha=0.8)
ax.fill_between(df['Date'], base_sales, sales, 
                 color='orange', alpha=0.3, label='Marketing Contribution')

ax.set_xlabel('Date', fontsize=12, fontweight='bold')
ax.set_ylabel('Sales Revenue ($)', fontsize=12, fontweight='bold')
ax.set_title('Sales Decomposition: Base vs Marketing Effects', 
             fontsize=14, fontweight='bold', pad=20)
ax.legend(loc='upper left')
ax.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print(f"\nBase Sales Summary:")
print(f"  Mean: ${base_sales.mean():,.0f}")
print(f"  Std: ${base_sales.std():,.0f}")
print(f"\nMarketing Contribution:")
contribution = sales - base_sales
print(f"  Mean weekly contribution: ${contribution.mean():,.0f}")
print(f"  % of total sales: {(contribution.mean() / sales.mean() * 100):.1f}%")

In [ ]:
# Extract the evolving base sales for the long-term model
base_sales = mmm.get_unobserved_component('trend')
base_sales.to_csv('../data/base_sales.csv')

# Plot the trend
base_sales.plot()